#### load the dataset


In [1]:
### path = data/dataset
from PIL import Image


In [14]:
#open an image file from the dataset
img = Image.open("../data/dataset/TRAIN/0/0_1.png")

#display the image
# img.show()

#show image pixels
print(img.size)

#convert image to rgba and print each pixel
img = img.convert('RGBA')
print(img.size)

#get the pixel values of the image
pixels = list(img.getdata())
binary_array = []
for pixel in pixels:
    if pixel[0] == 255 and pixel[1] == 255 and pixel[2] == 255:
        binary_array.append(0)
    else:
        binary_array.append(1)

#print all the non 0 pixels
print(binary_array)




(120, 125)
(120, 125)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 254, 255, 255)
(255, 254, 255, 255)
(255, 254, 255, 255)
(255, 254, 255, 255)
(255, 254, 255, 255)
(255, 255, 255, 255)
(255, 255, 255, 255)
(255, 255, 2